In [ ]:
# Learning material: https://github.com/openai/openai-cookbook/blob/main/examples/Assistants_API_overview_python.ipynb

In [1]:
from openai import OpenAI
import os
import json
from dotenv import load_dotenv
load_dotenv()

# pretty printing helper
import json
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [2]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-3.5-turbo",
)
show_json(assistant)

{'id': 'asst_qBVo1KzOYj0CXzzX4setXfn4',
 'created_at': 1712858606,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': []}

In [4]:
thread = client.beta.threads.create()
show_json(thread)


{'id': 'thread_ns41GshF1dChldMSStQI5Js5',
 'created_at': 1712858608,
 'metadata': {},
 'object': 'thread'}

In [5]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)
show_json(message)


{'id': 'msg_Vqq8x3KBJ9HdgIFl3eahKPCU',
 'assistant_id': None,
 'completed_at': None,
 'content': [{'text': {'annotations': [],
    'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
   'type': 'text'}],
 'created_at': 1712858611,
 'file_ids': [],
 'incomplete_at': None,
 'incomplete_details': None,
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'status': None,
 'thread_id': 'thread_ns41GshF1dChldMSStQI5Js5'}

In [6]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
show_json(run)

{'id': 'run_b102dsnFW2zXmNyX3H2HXTgO',
 'assistant_id': 'asst_qBVo1KzOYj0CXzzX4setXfn4',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1712858613,
 'expires_at': 1712859213,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_ns41GshF1dChldMSStQI5Js5',
 'tools': [],
 'usage': None,
 'temperature': 1.0}

In [7]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [8]:
run = wait_on_run(run, thread)
show_json(run)

{'id': 'run_b102dsnFW2zXmNyX3H2HXTgO',
 'assistant_id': 'asst_qBVo1KzOYj0CXzzX4setXfn4',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1712858613,
 'expires_at': None,
 'failed_at': 1712858619,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': {'code': 'rate_limit_exceeded',
  'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.'},
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1712858613,
 'status': 'failed',
 'thread_id': 'thread_ns41GshF1dChldMSStQI5Js5',
 'tools': [],
 'usage': {'completion_tokens': 0, 'prompt_tokens': 0, 'total_tokens': 0},
 'temperature': 1.0}

In [9]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_Vqq8x3KBJ9HdgIFl3eahKPCU',
   'assistant_id': None,
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
     'type': 'text'}],
   'created_at': 1712858611,
   'file_ids': [],
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
   'status': None,
   'thread_id': 'thread_ns41GshF1dChldMSStQI5Js5'}],
 'object': 'list',
 'first_id': 'msg_Vqq8x3KBJ9HdgIFl3eahKPCU',
 'last_id': 'msg_Vqq8x3KBJ9HdgIFl3eahKPCU',
 'has_more': False}